# Config stuff

In [2]:
from numpy.ma.core import inner

import ConnectionConfig as cc
from delta import DeltaTable
cc.setupEnvironment()

# Start local cluster

In [3]:
spark = cc.startLocalCluster("FACT_RIDES")
spark.getActiveSession()

# Make tables available

In [5]:
#EXTRACT
# Dimension date
dim_date = spark.read.format("delta").load("spark-warehouse/dimdate")

# Dimension weather
dim_weather = spark.read.format("delta").load("spark-warehouse/dimweather")

# Dimension customer
dim_customer = spark.read.format("delta").load("spark-warehouse/dimuser")

# Dimension lock
dim_lock = spark.read.format("delta").load("spark-warehouse/dimlock")

# Fact rides
fact_rides = spark.read.format("delta").load("spark-warehouse/factrides")

# Make tempviews from tables
dim_date.createOrReplaceTempView("dimDate")
dim_weather.createOrReplaceTempView("dimWeather")
dim_customer.createOrReplaceTempView("dimCustomer")
dim_lock.createOrReplaceTempView("dimLock")
fact_rides.createOrReplaceTempView("factRides")

# Analysis queries

## Student 1

### Wat zijn de drukke momenten (op dagbasis) in de week t.o.v. het weekend?

In [6]:
spark.sql("select dd.DayOfWeekStartMonday, count(fr.count_MV) as CountedRides from factRides fr \
          inner join dimDate dd on fr.date_SK = dd.date_SK \
          group by dd.DayOfWeekStartMonday \
          order by dd.DayOfWeekStartMonday asc ").show()

+--------------------+------------+
|DayOfWeekStartMonday|CountedRides|
+--------------------+------------+
|                   1|      523145|
|                   2|      549040|
|                   3|      395212|
|                   4|      311037|
|                   5|      304584|
|                   6|      392996|
|                   7|      538786|
+--------------------+------------+



In de week zijn vooral maandag en dinsdag de drukste dagen. Daarna neemt het aantal rides af elke dag.
In het weekend neemt het aantal rides terug toe en is het op zijn drukst op zondag.

### Hebben datumparameters invloed op de afgelegde afstand?

### Heeft weer invloed op ritten?

Op welke eigenschappen van de ritten kan het weer een invloed hebben:
1) Op aantal afgelegde ritten in een bepaald weertype
2) Op de gemiddelde duurtijd van een rit in een bepaald weertype
3) Op de gemiddelde afgelegde afstand van een rit in een bepaald weertype

In [30]:
spark.sql("select dw.weather_type, count(count_MV) as CountedRides, \
          round(sum(rideDuration_MV)/count(count_MV)) as AverageRideDuration \
          from factRides fr \
          inner join dimWeather dw on fr.weather_SK = dw.weather_SK \
          group by dw.weather_type \
          having dw.weather_type != 'weertype onbekend'").show()

+------------+------------+-------------------+
|weather_type|CountedRides|AverageRideDuration|
+------------+------------+-------------------+
|    Neutraal|         208|              677.0|
|   Aangenaam|         197|              718.0|
| Onaangenaam|          90|              635.0|
+------------+------------+-------------------+



#### 1. Invloed van weer op aantal afgelegde ritten
Op basis van de analyse hierboven zouden we kunnen zeggen dat bij neutraal of aangenaam weer er de meeste ritten worden afgewerkt.

#### 2. Invloed van weer op de duurtijd van een rit
We kunnen uit de analyse afleiden dat gebruikers bij aangenaam weer gemiddeld gezien de langste ritten afleggen op vlak van duurtijd. Bij onaangenaam weer zitten gebruikers het minst lang op hun voertuig.

In praktijk weten we echter dat we random weer situaties hebben toegevoegd aan onze data. Dus de patronenen die we hier zien zijn louter toeval.

#### 3. Invloed van weer op de afgelegde afstand

### Stijgt de populariteit van deelfietsen in Antwerpen met de jaren?

In [19]:
spark.sql("select dd.CalendarYear as Year, count(fr.count_MV) as CountedRides from factRides fr \
          inner join dimDate dd on fr.date_SK = dd.date_SK \
          group by dd.CalendarYear").show()

+----+------------+
|Year|CountedRides|
+----+------------+
|2019|      288379|
|2021|     1013666|
|2020|      958186|
|2022|      958269|
|2023|      705224|
+----+------------+



### Hebben de seizoenen invloed op de ritten?

1. Hoeveel ritten zijn er per seizoen?
2. Wat is de gemiddelde duurtijd per rit per seizoen?
3. Wat is de gemiddelde afgelegde afstand per rit per seizoen

In [34]:
spark.sql("select CASE \
          when (dd.MonthOfYear = 12 and dd.DayOfMonth >=21) or (dd.MonthOfYear in (1,2)) or (dd.MonthOfYear = 3 and dd.DayOfMonth <21) then 'winter' \
          when (dd.MonthOfYear = 3 and dd.DayOfMonth >=21) or (dd.MonthOfYear in (4,5)) or (dd.MonthOfYear = 6 and dd.DayOfMonth <21) then 'spring' \
          when (dd.MonthOfYear = 6 and dd.DayOfMonth >=21) or (dd.MonthOfYear in (7,8)) or (dd.MonthOfYear = 9 and dd.DayOfMonth <21) then 'summer' \
          when (dd.MonthOfYear = 9 and dd.DayOfMonth >=21) or (dd.MonthOfYear in (10,11)) or (dd.MonthOfYear = 12 and dd.DayOfMonth <21) then 'autumn' \
          end as season, \
          count(fr.count_MV) as CountedRides, \
          round(sum(fr.rideDuration_MV)/count(fr.count_MV)) as AverageRideDuration \
          from factRides fr \
          inner join dimDate dd on fr.date_SK = dd.date_SK \
          group by \
          CASE \
          when (dd.MonthOfYear = 12 and dd.DayOfMonth >=21) or (dd.MonthOfYear in (1,2)) or (dd.MonthOfYear = 3 and dd.DayOfMonth <21) then 'winter' \
          when (dd.MonthOfYear = 3 and dd.DayOfMonth >=21) or (dd.MonthOfYear in (4,5)) or (dd.MonthOfYear = 6 and dd.DayOfMonth <21) then 'spring' \
          when (dd.MonthOfYear = 6 and dd.DayOfMonth >=21) or (dd.MonthOfYear in (7,8)) or (dd.MonthOfYear = 9 and dd.DayOfMonth <21) then 'summer' \
          when (dd.MonthOfYear = 9 and dd.DayOfMonth >=21) or (dd.MonthOfYear in (10,11)) or (dd.MonthOfYear = 12 and dd.DayOfMonth <21) then 'autumn' \
          end \
          order by CountedRides desc").show()

+------+------------+-------------------+
|season|CountedRides|AverageRideDuration|
+------+------------+-------------------+
|autumn|      993222|              609.0|
|summer|      992891|              603.0|
|spring|      988528|              603.0|
|winter|      949083|              603.0|
+------+------------+-------------------+



1. We zien dat er in de herfst de meeste ritten zijn. Al is het verschil met de andere seizoenen beperkt.
2. de gemiddelde duurtijd per rit is in de herfst het hoogst. Bij de andere seizoenen is deze gelijk.

## Student 2

### Wat is de invloed van de woonplaats van de gebruikers op het gebruik van de vehicles?

In [39]:
spark.sql("select dc.city as user_city,\
          count(fr.ride_id) as total_rides,\
          round(AVG(fr.rideDuration_MV),1) AS avg_ride_duration_sec,\
          round(AVG(fr.distance_km_mv),3) AS avg_distance_km\
          from factRides fr\
          left outer join dimCustomer dc on dc.user_sk = fr.user_sk\
          group by dc.city\
          order by total_rides desc").show()


+--------------------+-----------+---------------------+---------------+
|           user_city|total_rides|avg_ride_duration_sec|avg_distance_km|
+--------------------+-----------+---------------------+---------------+
|           Antwerpen|     603708|                557.2|          2.405|
|             Schoten|     149514|                602.3|          2.581|
|Beveren-Waas/Haas...|     147219|                602.9|          2.581|
| Wilrijk (Antwerpen)|     132162|                651.0|          2.954|
|  Deurne (Antwerpen)|     123740|                693.4|          2.760|
|          Brasschaat|     109100|                601.4|          2.575|
|    Kapellen (Antw.)|     107046|                602.4|          2.583|
| Berchem (Antwerpen)|     104735|                575.6|          2.501|
|Broechem/Emblem/O...|     103617|                602.4|          2.579|
|s Gravenwezel/Sch...|     101869|                599.8|          2.568|
| Merksem (Antwerpen)|     100167|                6

### We willen voorspellen welke sloten preventief onderhoud nodig hebben. Bekijk hoe vaak slotnummers relatief gezien gebruikt worden.

In [71]:
spark.sql("select dl.lockid, (count(fr.start_lockid) + count(fr.end_lockid)) as CountLocks, \
          CountLocks * 1.0 / sum(count(fr.start_lockid) + count(fr.end_lockid)) over() as relative_usage\
          from factRides fr\
          left outer join dimLock dl on dl.lockid = fr.end_lockid\
          group by dl.lockid\
          order by CountLocks desc").show()

# Bij het starten van een rit moet het slot opengaan dus we moeten zowel start als eind slot in rekening brengen
# De sum() over() gaat ervoor zorgen dat hij de berekening gaat doen met alle data over een bepaalde partitie (in dit geval geen dus neemt hij alles)

+------+----------+------------------+
|lockid|CountLocks|    relative_usage|
+------+----------+------------------+
|  1548|     18530|0.0033467323033176|
|  2682|     18428|0.0033283099236663|
|   270|     18408|0.0033246976923622|
|  2772|     18374|0.0033185568991451|
|   738|     18318|0.0033084426514934|
|  3024|     18298|0.0033048304201892|
|  2538|     18296|0.0033044691970588|
|  3258|     18290|0.0033033855276675|
|   702|     18278|0.0033012181888850|
|  2754|     18254|0.0032968835113200|
|  2718|     18250|0.0032961610650592|
|   774|     18236|0.0032936325031463|
|  2825|     18224|0.0032914651643637|
|  4230|     18220|0.0032907427181029|
|  2358|     18162|0.0032802672473208|
|  2790|     18148|0.0032777386854079|
|  3168|     18132|0.0032748489003645|
|  3096|     18128|0.0032741264541037|
|   810|     18122|0.0032730427847125|
|  2988|     18096|0.0032683468840170|
+------+----------+------------------+
only showing top 20 rows



### Als een klant zijn abonnement stopzet, willen we kunnen voorspellen op welke stations dit het meeste effect zal hebben.

In [96]:
spark.sql("select dc.userid, dl.stationid, count(fr.ride_ID) as amount_rides \
          from factRides fr\
          left outer join dimCustomer dc on dc.user_sk = fr.user_sk\
          left outer join dimLock dl on dl.lockid = fr.start_lockid\
          where fr.start_lockid is not null\
          group by dc.userid, dl.stationid\
          order by amount_rides desc").show()

+------+---------+------------+
|userid|stationid|amount_rides|
+------+---------+------------+
| 37367|      162|          17|
|  5750|      180|          14|
| 24821|      162|          13|
| 46611|      205|          13|
| 41934|      294|          13|
| 38243|      180|          12|
|  7362|      160|          12|
| 24711|      180|          12|
| 38203|      180|          12|
| 17990|      180|          12|
|  7485|      161|          12|
| 26189|      180|          12|
| 19281|      278|          12|
| 41885|      161|          12|
| 10430|      161|          12|
|  6014|      180|          12|
| 33185|      294|          12|
| 47280|      161|          12|
| 35983|      162|          12|
| 33761|      161|          12|
+------+---------+------------+
only showing top 20 rows



In [97]:
# Extra zijn dit de ritten die geen start slot hebben (en dus ook geen station)
spark.sql("select dc.userid, count(fr.ride_ID) as amount_rides \
          from factRides fr\
          left outer join dimCustomer dc on dc.user_sk = fr.user_sk\
          left outer join dimLock dl on dl.lockid = fr.start_lockid\
          where fr.start_lockid is null\
          group by dc.userid\
          order by amount_rides desc").show()

+------+------------+
|userid|amount_rides|
+------+------------+
| 24213|          31|
| 45176|          26|
|  6027|          23|
| 34758|          23|
| 37660|          23|
| 38470|          23|
| 34805|          22|
| 38835|          22|
|  3197|          22|
|  7716|          22|
| 14229|          22|
| 32022|          21|
| 28331|          21|
| 37534|          21|
|  2201|          21|
| 57187|          21|
| 39947|          21|
|  2247|          20|
| 54381|          20|
| 23970|          20|
+------+------------+
only showing top 20 rows



### Extra vraag 1

### Extra vraag 2